In [1]:
from elasticsearch import Elasticsearch

In [2]:
client=Elasticsearch("http://127.0.0.1:9200")

In [3]:
client.ping()

True

In [4]:
client.indices.create(index="test_index")

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'test_index'})

<bound method IndicesClient.get_alias of <elasticsearch._sync.client.indices.IndicesClient object at 0x0000027162832490>>

In [8]:
documents = [
        {"id": "1", "document": {"name": "John Doe", "age": 30, "role": "Developer"}},
        {"id": "2", "document": {"name": "Jane Smith", "age": 25, "role": "Designer"}},
        {"id": "3", "document": {"name": "Alice Johnson", "age": 35, "role": "Manager"}},
        {"id": "4", "document": {"name": "Bob Brown", "age": 28, "role": "Developer"}}
]

In [9]:
for document in documents:
    client.index(index="test_index",id=document["id"],document=document["document"])

In [39]:
query = {
    "query": {
        "match": {
            "role": {
                "query": "devloper",
                "fuzziness": "AUTO"
            }
        }
    }
}

In [40]:
response = client.search(index="test_index", body=query)

In [41]:
for hit in response["hits"]["hits"]:
    print(hit["_source"])

{'name': 'John Doe', 'age': 30, 'role': 'Developer'}
{'name': 'Bob Brown', 'age': 28, 'role': 'Developer'}


IndexError: list index out of range

In [7]:
index_name="doctors_and_hospitals"

In [8]:
mappings={
    "mappings":{
        "properties":{
            "user_id":{"type":"keyword"},
            "name":{"type":"text"},
            "role":{"type":"text"},
            "speciality":{"type":"text"},
            "address":{"type":"text"},
            "location":{"type":"geo_point"}
            
        }
        
    }
}

In [9]:
client.indices.create(index=index_name,body=mappings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'doctors_and_hospitals'})

In [45]:
doctors = [
    {
        "user_id": "1",
        "name": "Dr. John Doe",
        "role": "Consultant",
        "speciality": "Cardiologist",
        "experience": 15,
        "address": "123 Heart Lane, Cityville",
        "location": {"lat": 40.7128, "lon": -74.0060}  # New York
    },
    {
        "user_id": "2",
        "name": "Dr. Alice Smith",
        "role": "Surgeon",
        "speciality": "Dermatologist",
        "experience": 10,
        "address": "456 Skin Road, Townsville",
        "location": {"lat": 34.0522, "lon": -118.2437}  # Los Angeles
    },
]


for doctor in doctors:
    client.index(index=index_name, document=doctor)
print("Data indexed.")


Data indexed.


In [47]:
query = {
    "query": {
        "multi_match": {
            "query": "Cardiologist",
            "fields": ["name", "speciality"],
            "fuzziness":"AUTO"
        }
    }
}

response = client.search(index=index_name, body=query)
print("Search Results:")
for hit in response["hits"]["hits"]:
    print(hit["_source"])

Search Results:
{'user_id': '1', 'name': 'Dr. John Doe', 'role': 'Consultant', 'speciality': 'Cardiologist', 'experience': 15, 'address': '123 Heart Lane, Cityville', 'location': {'lat': 40.7128, 'lon': -74.006}}


In [58]:
import random

# Helper function to generate random coordinates near Mumbai
def generate_coordinates(base_lat=19.0760, base_lon=72.8777, radius=0.1):
    lat = base_lat + random.uniform(-radius, radius)
    lon = base_lon + random.uniform(-radius, radius)
    return {"lat": lat, "lon": lon}

# Specialities and roles
specialities = ["Cardiologist", "Dermatologist", "Neurologist", "Pediatrician", "Orthopedic", "Gynecologist", "Oncologist"]
roles = ["Consultant", "Surgeon", "General Physician"]

# Generate 100 doctors
doctors = []
for i in range(5000):
    doctor = {
        "user_id": str(i),
        "name": f"Dr. {random.choice(['John', 'Alice', 'Raj', 'Priya', 'Sanjay', 'Anita', 'Vivek', 'Neha'])} {random.choice(['Sharma', 'Patel', 'Mehta', 'Gupta', 'Khan', 'Reddy', 'Nair'])}",
        "role": random.choice(roles),
        "speciality": random.choice(specialities),
        "experience": random.randint(1, 40),  # Random experience between 1 and 40 years
        "address": f"{random.randint(101, 999)} {random.choice(['Marine Drive', 'Carter Road', 'Bandra West', 'Andheri East', 'Colaba', 'Borivali', 'Juhu'])}, Mumbai",
        "location": generate_coordinates()
    }
    doctors.append(doctor)

# Print the first 5 doctors to preview
for doctor in doctors[:5]:
    print(doctor)


{'user_id': '1', 'name': 'Dr. Priya Reddy', 'role': 'Surgeon', 'speciality': 'Cardiologist', 'experience': 21, 'address': '622 Juhu, Mumbai', 'location': {'lat': 19.017034007135056, 'lon': 72.85451372727303}}
{'user_id': '2', 'name': 'Dr. Sanjay Patel', 'role': 'Consultant', 'speciality': 'Orthopedic', 'experience': 32, 'address': '685 Borivali, Mumbai', 'location': {'lat': 19.0644827514327, 'lon': 72.84550389973764}}
{'user_id': '3', 'name': 'Dr. John Nair', 'role': 'Consultant', 'speciality': 'Dermatologist', 'experience': 13, 'address': '816 Carter Road, Mumbai', 'location': {'lat': 19.12254247574048, 'lon': 72.79212064649903}}
{'user_id': '4', 'name': 'Dr. John Khan', 'role': 'Consultant', 'speciality': 'Orthopedic', 'experience': 13, 'address': '453 Andheri East, Mumbai', 'location': {'lat': 19.081210755009536, 'lon': 72.94315654779008}}
{'user_id': '5', 'name': 'Dr. Priya Patel', 'role': 'Surgeon', 'speciality': 'Gynecologist', 'experience': 26, 'address': '642 Borivali, Mumbai',

In [59]:
data=[d["location"] for d in doctors]

In [61]:
import pandas as pd

In [62]:
df=pd.DataFrame(data)

In [68]:
df.rename(columns={"lon":"lng"},inplace=True)

In [69]:
df.head()

,lat,lng
0,19.017034,72.854514
1,19.064483,72.845504
2,19.122542,72.792121
3,19.081211,72.943157
4,19.020429,72.843080


In [70]:
df.to_csv("locations.csv",index=False,encoding="utf-8")

In [71]:
index_name

'doctors'

In [74]:
for doctor in doctors:
    client.index(index=index_name,document=doctor)

In [97]:
geo_query={
    "size":20,
    "query":{
        "geo_distance":{
            "distance":"10km",
            "location":{"lat":19.16492,"lon":72.87133}
        }
    }
}

In [84]:
response=client.search(index=index_name,body=geo_query)

In [85]:
data=[]
for hit in response["hits"]["hits"]:
    print(hit["_source"])

In [86]:
data

[{'lat': 19.155790455540462, 'lon': 72.86973684472143},
 {'lat': 19.15321046996135, 'lon': 72.88763181260637},
 {'lat': 19.120908944146848, 'lon': 72.87470665809411},
 {'lat': 19.13859102627138, 'lon': 72.85072016745433},
 {'lat': 19.164906220225756, 'lon': 72.89281952745594},
 {'lat': 19.16074575513179, 'lon': 72.8271132026206},
 {'lat': 19.126672967169178, 'lon': 72.85848305923484},
 {'lat': 19.173964212525647, 'lon': 72.86346891014435},
 {'lat': 19.172494349895665, 'lon': 72.88630150443333},
 {'lat': 19.16043612795266, 'lon': 72.8556901246473},
 {'lat': 19.15163717096808, 'lon': 72.8273589910145}]

In [89]:
df=pd.DataFrame(data)

In [91]:
df.rename(columns={"lon":"lng"},inplace=True)

In [93]:
df.to_csv("locations_got.csv",index=False,encoding="utf-8")

In [107]:
locations=['Marine Drive', 'Carter Road', 'Bandra West', 'Andheri East', 'Colaba', 'Borivali', 'Juhu',"Girgaon","Mazgaon","nagpada","Gateway of India", "Marine Drive", "Chhatrapati Shivaji Maharaj Terminus", "Elephanta Caves",
    "Siddhivinayak Temple", "Haji Ali Dargah", "Hanging Gardens", "Juhu Beach", "Girgaon Chowpatty",
    "Bandra-Worli Sea Link", "Worli Fort", "Global Vipassana Pagoda", "Prince of Wales Museum",
    "Dr. Bhau Daji Lad Museum", "Mumbai Zoo (Veermata Jijabai Udyan)", "Nehru Science Centre",
    "ISKCON Temple", "Mount Mary Church", "Kanheri Caves", "Sanjay Gandhi National Park",
    "Mahalakshmi Racecourse", "Colaba Causeway", "Crawford Market", "Chor Bazaar",
    "Fashion Street", "High Street Phoenix", "Palladium Mall", "Phoenix Market City",
    "R City Mall", "Inorbit Mall", "Atria Mall", "Linking Road", "Hill Road", "Carter Road Promenade",
    "Powai Lake", "Vihar Lake", "Tulsi Lake", "Banganga Tank", "Mahim Fort", "Shivaji Park",
    "Rajabai Clock Tower", "Flora Fountain", "Asiatic Library", "Film City", "EsselWorld",
    "Water Kingdom", "Tikuji-ni-Wadi", "Kidzania Mumbai", "Hakone Entertainment Centre",
    "Snow World", "Mumbai Port Trust Garden", "Oval Maidan", "Azad Maidan", "Kala Ghoda",
    "Nariman Point", "Breach Candy", "Walkeshwar", "Mani Bhavan", "Antilia Building",
    "Aarey Milk Colony", "Borivali National Park", "Versova Beach", "Marve Beach", "Madh Island",
    "Gorai Beach", "Manori Beach", "Arnala Beach", "Chincholi Bunder", "Bandra Reclamation",
    "Lokhandwala Complex", "Andheri West", "Andheri East", "Vile Parle", "Santacruz", "Khar",
    "Dadar", "Parel", "Lower Parel", "Byculla", "Mazgaon", "Sion", "Matunga", "Chembur",
    "Kurla", "Mulund", "Bhandup", "Ghatkopar", "Vikhroli", "Powai", "Kanjurmarg", "Malad",
    "Kandivali", "Borivali", "Dahisar", "Mira Road", "Bhayandar", "Thane", "Kalwa", "Mumbra",
    "Navi Mumbai", "Panvel"]

In [108]:

for i in range(1000000):
    doctor = {
        "user_id": str(i),
        "name": f"Dr. {random.choice(['John', 'Alice', 'Raj', 'Priya', 'Sanjay', 'Anita', 'Vivek', 'Neha'])} {random.choice(['Sharma', 'Patel', 'Mehta', 'Gupta', 'Khan', 'Reddy', 'Nair'])}",
        "role": random.choice(roles),
        "speciality": random.choice(specialities),
        "experience": random.randint(1, 40),  # Random experience between 1 and 40 years
        "address": f"{random.randint(101, 999)} {random.choice(locations)}, Mumbai",
        "location": generate_coordinates()
    }
    client.index(index=index_name,document=doctor)

KeyboardInterrupt: 

In [95]:
len(doctors)

5000

In [96]:
for doctor in doctors:
    client.index(index=index_name,document=doctor)

In [99]:
from time import time

In [110]:
t=time()
response=client.search(index=index_name,body=geo_query)
print(time()-t)

0.025182723999023438


In [114]:
query = {
    "query": {
        "multi_match": {
            "query": "viek",
            "fields": ["name", "speciality","address"],
            "fuzziness":"AUTO"
        }
    }
}
t=time()
response = client.search(index=index_name, body=query)
print("Search Results:")
print(time()-t)
for hit in response["hits"]["hits"]:
    print(hit["_source"])

Search Results:
0.009149312973022461
{'user_id': '12', 'name': 'Dr. Vivek Khan', 'role': 'Surgeon', 'speciality': 'Dermatologist', 'experience': 27, 'address': '435 Linking Road, Mumbai', 'location': {'lat': 19.114573101227887, 'lon': 72.89007566790876}}
{'user_id': '28', 'name': 'Dr. Vivek Mehta', 'role': 'Consultant', 'speciality': 'Gynecologist', 'experience': 37, 'address': '200 Shivaji Park, Mumbai', 'location': {'lat': 19.140321081069118, 'lon': 72.97344011305786}}
{'user_id': '30', 'name': 'Dr. Vivek Nair', 'role': 'Surgeon', 'speciality': 'Oncologist', 'experience': 34, 'address': '811 Kurla, Mumbai', 'location': {'lat': 19.144837119966812, 'lon': 72.85628887175777}}
{'user_id': '33', 'name': 'Dr. Vivek Mehta', 'role': 'Consultant', 'speciality': 'Gynecologist', 'experience': 36, 'address': '773 Shivaji Park, Mumbai', 'location': {'lat': 19.172556944071744, 'lon': 72.94158178394083}}
{'user_id': '36', 'name': 'Dr. Vivek Nair', 'role': 'Surgeon', 'speciality': 'Orthopedic', 'exp

In [ ]:
['Marine Drive', 'Carter Road', 'Bandra West', 'Andheri East', 'Colaba', 'Borivali', 'Juhu',"Girgaon","Mazgaon","nagpada","Gateway of India", "Marine Drive", "Chhatrapati Shivaji Maharaj Terminus", "Elephanta Caves",
    "Siddhivinayak Temple", "Haji Ali Dargah", "Hanging Gardens", "Juhu Beach", "Girgaon Chowpatty",
    "Bandra-Worli Sea Link", "Worli Fort", "Global Vipassana Pagoda", "Prince of Wales Museum",
    "Dr. Bhau Daji Lad Museum", "Mumbai Zoo (Veermata Jijabai Udyan)", "Nehru Science Centre",
    "ISKCON Temple", "Mount Mary Church", "Kanheri Caves", "Sanjay Gandhi National Park",
    "Mahalakshmi Racecourse", "Colaba Causeway", "Crawford Market", "Chor Bazaar",
    "Fashion Street", "High Street Phoenix", "Palladium Mall", "Phoenix Market City",
    "R City Mall", "Inorbit Mall", "Atria Mall", "Linking Road", "Hill Road", "Carter Road Promenade",
    "Powai Lake", "Vihar Lake", "Tulsi Lake", "Banganga Tank", "Mahim Fort", "Shivaji Park",
    "Rajabai Clock Tower", "Flora Fountain", "Asiatic Library", "Film City", "EsselWorld",
    "Water Kingdom", "Tikuji-ni-Wadi", "Kidzania Mumbai", "Hakone Entertainment Centre",
    "Snow World", "Mumbai Port Trust Garden", "Oval Maidan", "Azad Maidan", "Kala Ghoda",
    "Nariman Point", "Breach Candy", "Walkeshwar", "Mani Bhavan", "Antilia Building",
    "Aarey Milk Colony", "Borivali National Park", "Versova Beach", "Marve Beach", "Madh Island",
    "Gorai Beach", "Manori Beach", "Arnala Beach", "Chincholi Bunder", "Bandra Reclamation",
    "Lokhandwala Complex", "Andheri West", "Andheri East", "Vile Parle", "Santacruz", "Khar",
    "Dadar", "Parel", "Lower Parel", "Byculla", "Mazgaon", "Sion", "Matunga", "Chembur",
    "Kurla", "Mulund", "Bhandup", "Ghatkopar", "Vikhroli", "Powai", "Kanjurmarg", "Malad",
    "Kandivali", "Borivali", "Dahisar", "Mira Road", "Bhayandar", "Thane", "Kalwa", "Mumbra",
    "Navi Mumbai", "Panvel"]

In [109]:
client.count(index=index_name)

ObjectApiResponse({'count': 130528, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}})

In [ ]:
indices = client.cat.indices(format="json")
for index in indices:
    print(index["index"], index["docs.count"], index["store.size"])
